In [ ]:
import os
from io import BytesIO

import h5py
import requests
import pandas as pd
from PIL import Image
from torchvision import transforms

In [ ]:
data_path = "data"

path_test_df = os.path.join(data_path, "test-metadata.csv")
path_test_hdf5 = os.path.join(data_path, "test-image.hdf5")

In [ ]:
test_df = pd.read_csv(path_test_df)
isic_id = test_df.isic_id.values.tolist()

hdf5_img = h5py.File(path_test_hdf5, 'r')

In [ ]:
example_image = Image.open(BytesIO(hdf5_img[isic_id[0]][()]))
example_image.show()

In [ ]:
def prepare_request_body(img: Image) -> dict:
    transformations = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    image = transformations(img)[None, :]
    
    body = {
      "inputs": [
        {
          "name": "input.1",
          "shape": [1, 3, 224, 224],
          "datatype": "FP32",
          "data": image.tolist()
        }
      ]
    }
    
    return body

In [ ]:
URL = "http://skin-cancer-detection.kubeflow-user-example-com.svc.cluster.local/v2/models/skin_cancer_detection/infer"
response = requests.post(URL, json=prepare_request_body(example_image))

In [ ]:
status_code = response.status_code

if status_code == 200:
    prob = response.json()["outputs"][0]["data"][0]
    print(f"The probability that the lesion is malignant is {prob:.3f}")
else:
    print(f"Something went wrong. Status Code: {status_code}")
    print(response.text)